<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://files.oproject.org/tmvalogo.png" height="50%" width="50%">

# TMVA  Reader Example 

#### Example of applying the trained classified of TMVA on a data set and evaluate the classifier

In [1]:
import ROOT
from ROOT import TMVA

Welcome to JupyROOT 6.12/06


### Enable JS visualization
To use new interactive features in notebook we have to enable a module called JsMVA. This can be done by using ipython magic: %jsmva.

In [19]:
from IPython.core.extensions import ExtensionManager
ExtensionManager(get_ipython()).load_extension("JsMVA.JsMVAMagic")

In [2]:
%jsroot on

### Choose TMVA method to evaluate

In [3]:
methodName = "BDT"

###  Give input file 


In [4]:
inputFile = ROOT.TFile("Higgs_data.root")

In [5]:
inputFile.ls()
#inputFile.sig_tree.Print()

TFile**		Higgs_data.root	
 TFile*		Higgs_data.root	
  KEY: TTree	sig_tree;1	tree
  KEY: TTree	bkg_tree;1	tree


### Declare Reader


In [6]:
TMVA.Tools.Instance()

reader = TMVA.Reader( "!Color:!Silent" )


### Add Variable

We add variables to the reader exactly in the same way we did for the **DataLoader** during the training

We need to specify the address of the variable in order to pass it to TMVA when we iterate on the TTree


In [7]:
from array import array
m_jj = array('f',[0])
m_jjj = array('f',[0])
m_lv = array('f',[0])
m_jlv = array('f',[0])
m_bb = array('f',[0])
m_wbb = array('f',[0])
m_wwbb = array('f',[0])

In [8]:
#add variables 
reader.AddVariable("m_jj",m_jj)
reader.AddVariable("m_jjj",m_jjj)
reader.AddVariable("m_lv",m_lv)
reader.AddVariable("m_jlv",m_jlv)
reader.AddVariable("m_bb",m_bb)
reader.AddVariable("m_wbb",m_wbb)
reader.AddVariable("m_wwbb",m_wwbb)


## Setup Classifiers

We set up the classifiers by reading the input weights from the appropriate files

The file is stored for example as 

####  *dataset/weights/TMVAClassification_BDT.weights.xml

In [9]:
weightfile = "dataset/weights/TMVAClassification_" + methodName + ".weights.xml"

In [10]:
reader.BookMVA( methodName, weightfile );

                         : Booking "BDT" of type "BDT" from dataset/weights/TMVAClassification_BDT.weights.xml.
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "BDT" of type: "BDT"


### Book histogram to visualize the response

In [11]:
h1 = ROOT.TH1D("h1","Classifier Output on Background Events",100,-0.5,0.5)
h2 = ROOT.TH1D("h2","Classifier Output on Signal Events",100,-0.5,0.5)

## Iterate on the events from the input TTree 

### Evaluation of the classifier on the given input events

We iterate on the input event in the given TTree. We provide as input first the background tree 

In [12]:
ievt = 0

for entry in inputFile.bkg_tree:
    
    m_jj[0] = entry.m_jj
    m_jjj[0] = entry.m_jjj
    m_lv[0] = entry.m_lv
    m_jlv[0] = entry.m_jlv
    m_bb[0] = entry.m_bb
    m_wbb[0] = entry.m_wbb
    m_wwbb[0] = entry.m_wwbb
    
    output = reader.EvaluateMVA(methodName)
  
    h1.Fill(output)
    
    
    if (ievt%10000)==0 : print 'Event ',ievt,'m_jj=',m_jj[0],'MVA output =',output
    ievt += 1
#    if (ievt > 20000) : break
    

Event  0 m_jj= 0.946652472019 MVA output = 0.159885163392


In [13]:
h1.Draw()
ROOT.gPad.Draw()

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


### Evaluation on the signal  events

We evaluate now the classifier on the another tree (e.g. the signal events)



In [14]:
ievt = 0

for entry in inputFile.sig_tree:

    m_jj[0] = entry.m_jj
    m_jjj[0] = entry.m_jjj
    m_lv[0] = entry.m_lv
    m_jlv[0] = entry.m_jlv
    m_bb[0] = entry.m_bb
    m_wbb[0] = entry.m_wbb
    m_wwbb[0] = entry.m_wwbb
    
    output = reader.EvaluateMVA(methodName)
  
    h2.Fill(output)
    
    if (ievt%10000)==0 : print 'Event ',ievt,'m_jj=',m_jj[0],'MVA output =',output
    ievt += 1
#    if (ievt > 10000) : break
    

Event  0 m_jj= 1.35376000404 MVA output = 0.0325890029293


In [15]:
h2.SetLineColor(ROOT.kRed)

In [16]:
h2.Draw("SAME")
ROOT.gPad.Draw()
ROOT.gPad.BuildLegend()

<ROOT.TLegend object ("TPave") at 0xd185890>